In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["VLLM_USE_V1"] = "0"

In [2]:
from vllm import LLM, SamplingParams

# llm = LLM(model="/mnt/nfs-data/kilm-storage/public-llm/Qwen2.5-3B-Instruct", gpu_memory_utilization=0.95)
# llm = LLM(model="/mnt/nfs-data/kilm-storage/public-llm/Qwen2.5-3B-Instruct", gpu_memory_utilization=0.9)

llm = LLM(
    model="/data01/kilm/users/quocvh/HF/models/Qwen/Qwen3-4B-Instruct-2507",
    gpu_memory_utilization=0.85,
    max_model_len=31000,
    enforce_eager=True
)

Skipping import of cpp extensions due to incompatible torch version 2.7.1+cu126 for torchao version 0.14.0         Please see GitHub issue #2919 for more info


INFO 11-05 17:45:53 [__init__.py:235] Automatically detected platform cuda.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-05 17:46:04 [config.py:1604] Using max model len 31000
WARNING 11-05 17:46:04 [cuda.py:103] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 11-05 17:46:04 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='/data01/kilm/users/quocvh/HF/models/Qwen/Qwen3-4B-Instruct-2507', speculative_config=None, tokenizer='/data01/kilm/users/quocvh/HF/models/Qwen/Qwen3-4B-Instruct-2507', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=31000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='xgrammar', disable_fallback=False, disable_any_whitespace=False, d

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 11-05 17:46:10 [default_loader.py:262] Loading weights took 1.56 seconds
INFO 11-05 17:46:11 [model_runner.py:1115] Model loading took 7.6065 GiB and 4.705542 seconds
INFO 11-05 17:46:17 [worker.py:295] Memory profiling takes 5.12 seconds
INFO 11-05 17:46:17 [worker.py:295] the current vLLM instance can use total_gpu_memory (23.50GiB) x gpu_memory_utilization (0.85) = 19.97GiB
INFO 11-05 17:46:17 [worker.py:295] model weights take 7.61GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 2.29GiB; the rest of the memory reserved for KV Cache is 10.02GiB.
INFO 11-05 17:46:17 [executor_base.py:113] # cuda blocks: 4561, # CPU blocks: 1820
INFO 11-05 17:46:17 [executor_base.py:118] Maximum concurrency for 31000 tokens per request: 2.35x
INFO 11-05 17:46:25 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 14.04 seconds


In [3]:
import pandas as pd

# SET = "private"
SET = "public"
# df = pd.read_csv("/data01/kilm/users/quocvh/VAR/var_pdf/data/public_test_input/question.csv")
# df = pd.read_csv(f"/data01/kilm/users/quocvh/VAR/var_pdf/submit2/qa_{SET}_search.csv")
# df = pd.read_csv(f"/data01/kilm/users/quocvh/VAR/var_pdf/submit4/qa_{SET}_search_v2.csv")
# df = pd.read_csv(f"/data01/kilm/users/quocvh/VAR/var_pdf/submit4/qa_{SET}_search_v6.csv")
df = pd.read_csv(f"/data01/kilm/users/quocvh/VAR/var_pdf/submit4/qa_{SET}_search_v7.csv")

# df.head()

In [4]:
data = df.to_dict("records")

In [5]:
len(data)

310

In [6]:

TEMPLATE =  """Hãy chọn các đáp án đúng cho câu hỏi trắc nghiệm sau
Ouput format: Các đáp án đúng cách nhau bởi dấu ",", eg: A,B,C,D
Chỉ trả về các chữ cái ABCD, không trả kèm gì thêm

Câu hỏi: {Question}
A. {A}
B. {B}
C. {C}
D. {D}
"""

TEMPLATE =  """Hãy chọn các đáp án đúng cho câu hỏi trắc nghiệm sau dựa vào tài liệu
Ouput format: Các đáp án đúng cách nhau bởi dấu ",", eg: A,B,C,D
Chỉ trả về các chữ cái ABCD, không trả kèm gì thêm

Tài liệu: {context}

Câu hỏi: {Question}
A. {A}
B. {B}
C. {C}
D. {D}
"""

TEMPLATE =  """Tài liệu: {context}

Hãy chọn các đáp án đúng cho câu hỏi trắc nghiệm sau dựa vào tài liệu
Ouput format: Các đáp án đúng cách nhau bởi dấu ",", eg: A,B,C,D
Chỉ trả về các chữ cái ABCD, không trả kèm gì thêm

Câu hỏi: {Question}
A. {A}
B. {B}
C. {C}
D. {D}
"""

TEMPLATE =  """Tài liệu: {context}

Hãy chọn các đáp án đúng cho câu hỏi trắc nghiệm sau dựa vào tài liệu
Ouput format: 
Please show your choice in the answer field with only the choice letters, e.g., "C" or "A,D"

Câu hỏi: {Question}
A. {A}
B. {B}
C. {C}
D. {D}

Đáp án đúng là:
"""


x = data[0]


# print(TEMPLATE.format(**x))

In [7]:
tokenizer = llm.get_tokenizer()

In [8]:
# tokenizer.decode(tokenizer.encode("hello"))

In [9]:
MM = []
for x in data:
    x["context"] = tokenizer.decode(tokenizer.encode(x["context"])[:30000])
    prompt = TEMPLATE.format(**x)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]
    MM.append(messages)


sampling_params = SamplingParams(temperature=0.0, top_k=1, max_tokens=10)
outputs = llm.chat(MM, sampling_params)

INFO 11-05 17:46:28 [chat_utils.py:473] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Adding requests:   0%|          | 0/310 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/310 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [14]:
preds = []
for x, d in zip(outputs, data):
    res = x.outputs[0].text
    answers = []
    for a in res.split(","):
        a = a.split(".")[0].strip()
        
        
        if a:
            a = a[0]
        else:
            continue

        if a in "ABCD":
            answers.append(a)

    print(res, answers)


    
    if not answers:
        preds.append("A")
        d["pred"] = None
    else:
        preds.append(",".join(answers))
        d["pred"] = ",".join(answers)


Không có thông tin nào trong tài liệu được c []
B ['B']
C ['C']
B ['B']
C ['C']
A ['A']
B ['B']
A ['A']
A ['A']
D ['D']
B ['B']
C ['C']
A ['A']
A,D ['A', 'D']
Không có thông tin trong tài liệu cung cấp []
D ['D']
B ['B']
C ['C']
B ['B']
A ['A']
C ['C']
C ['C']
C ['C']
A ['A']
A ['A']
B ['B']
A ['A']
B ['B']
A ['A']
D ['D']
B ['B']
B ['B']
B ['B']
D ['D']
B

Giải thích:  
- Tín ['B']
A ['A']
Không có thông tin trong tài liệu về việc sử []
B ['B']
A ['A']
A ['A']
A,D ['A', 'D']
A ['A']
A ['A']
B ['B']
Không có thông tin nào trong tài liệu về kích []
Câu hỏi yêu cầu nội suy giá trị ['C']
B ['B']
B ['B']
B ['B']
B ['B']
A ['A']
Không có thông tin trong tài liệu cung cấp []
A ['A']
A ['A']
A ['A']
A ['A']
Không có thông tin nào trong tài liệu về môn []
C ['C']
B ['B']
C ['C']
Không có thông tin nào trong tài liệu về " []
A ['A']
Không có thông tin nào trong tài liệu về môn []
B ['B']
A ['A']
A,D ['A', 'D']
B ['B']
C ['C']
B ['B']
B ['B']
A ['A']
Không có thông tin nào trong tài liệu được c 

In [20]:
import glob

md_paths = sorted(glob.glob(f"/data01/kilm/users/quocvh/VAR/var_pdf/submit4/pymuf4llm_{SET}_output/*/*.md"))
md_docs = {}
for i, path in enumerate(md_paths):
    with open(path) as f:
        text = f.read()

    # print(len(text.split()))

    idx = int(path.split("/")[-2].split("_")[-1])
    # idx2i[idx] = i
    md_docs[idx] = text

In [57]:
import json
import ast

na_data = [x for x in data if not x["pred"] ]
len(na_data)
for x in na_data:
    # print(x["search"])
    idxs =  ast.literal_eval(x["search"])
    # print(idxs)
    if len(idxs) <= 1:
        x["pred"] = "A"
        continue
    
    idx =  int(idxs[9].split("_")[-1])
    # print(idx)
    context = md_docs[idx]
    x["context"] = context

na_data = [x for x in data if not x["pred"] ]
len(na_data)

6

In [58]:
MM = []
for x in na_data:
    x["context"] = tokenizer.decode(tokenizer.encode(x["context"])[:30000])
    prompt = TEMPLATE.format(**x)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]
    MM.append(messages)


sampling_params = SamplingParams(temperature=0.0, top_k=1, max_tokens=10)
outputs = llm.chat(MM, sampling_params)

Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [59]:
# preds = []
for x, d in zip(outputs, na_data):
    res = x.outputs[0].text
    answers = []
    for a in res.split(","):
        a = a.split(".")[0].strip()
        
        
        if a:
            a = a[0]
        else:
            continue

        if a in "ABCD":
            answers.append(a)

    print(res, answers)
    
    if not answers:
        # preds.append("A")
        d["pred"] = None
    else:
        # preds.append(",".join(answers))
        d["pred"] = ",".join(answers)


Không có thông tin nào trong tài liệu cung []
Không có thông tin nào trong tài liệu về môn []
Không có thông tin nào trong tài liệu về " []
Không có thông tin nào trong tài liệu được c []
Không có thông tin nào trong tài liệu về " []
Không có thông tin nào trong tài liệu cung []


In [60]:
# na_data[0]

preds = []
for x in data:
    if x["pred"]:
        preds.append(x["pred"])
    else:
        preds.append("A")
len(preds)

310

In [61]:
df["num_correct"] = [len(x.split(",")) for x in preds]
df["answers"] = preds
# df.to_csv(f"/data01/kilm/users/quocvh/VAR/var_pdf/submit2/qa_{SET}.csv", encoding="utf-8-sig", index=False)
df.to_csv(f"/data01/kilm/users/quocvh/VAR/var_pdf/submit4/qa_{SET}_v10.csv", encoding="utf-8-sig", index=False)